In [15]:
#required imports
from sklearn.neural_network import MLPRegressor
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error  
from math import sqrt 
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from scipy import stats

In [16]:
#load features
features = pd.read_csv("features.csv")
features.head()

,Day,Month,Season,Weekday,Holiday,Daily Weather,Daily Weather (Past),Humidity,Humidity (Past),Windspeed,Windspeed (Past),Apparent Temperature (Avg),Apparent Temperature (Avg) (Past),Rented Bikes,Rented Bikes (Future)
0,1,January,Winter,Monday,False,partly-cloudy-day,fog,0.88,0.94,1.59,0.55,46.740,36.295,281,279
1,2,January,Winter,Tuesday,False,partly-cloudy-day,partly-cloudy-day,0.86,0.88,2.07,1.59,42.150,46.740,279,274
2,3,January,Winter,Wednesday,False,clear-day,partly-cloudy-day,0.86,0.86,4.13,2.07,45.450,42.150,274,161
3,4,January,Winter,Thursday,False,rain,clear-day,0.87,0.86,3.60,4.13,46.200,45.450,161,270
4,5,January,Winter,Friday,False,partly-cloudy-day,rain,0.81,0.87,7.43,3.60,56.085,46.200,270,62


In [17]:
#transorm non-numerical labels to numerical labels
le = LabelEncoder()
features.loc[:,'Holiday'] = le.fit_transform(features['Holiday'])

In [18]:

cleanup_nums = {"Month":   {"January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6, "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December": 12},
               "Season": {"Spring": 1, "Summer": 2, "Autumn": 3, "Winter": 4},
               "Weekday": {"Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4,
                                  "Friday": 5, "Saturday": 6, "Sunday":7 },
               "Daily Weather": {"clear-day": 1, "partly-cloudy-day": 2,"cloudy":3,"wind":4, "rain": 5, "fog": 6, "snow": 7},
               "Daily Weather (Past)": {"clear-day": 1, "partly-cloudy-day": 2,"cloudy":3,"wind":4, "rain": 5,"fog": 6, "snow": 7} }

In [19]:
features.replace(cleanup_nums, inplace=True)

In [20]:
#target variable
y = features['Rented Bikes']
len(y)

1514

In [21]:
#Feature Matrix with past and today data
X=features.loc[:,['Day','Month','Season','Weekday','Daily Weather','Daily Weather (Past)','Humidity','Humidity (Past)','Windspeed', 'Windspeed (Past)', 'Apparent Temperature (Avg)','Apparent Temperature (Avg) (Past)','Rented Bikes (Future)']]
X.head()

,Day,Month,Season,Weekday,Daily Weather,Daily Weather (Past),Humidity,Humidity (Past),Windspeed,Windspeed (Past),Apparent Temperature (Avg),Apparent Temperature (Avg) (Past),Rented Bikes (Future)
0,1,1,4,1,2,6,0.88,0.94,1.59,0.55,46.740,36.295,279
1,2,1,4,2,2,2,0.86,0.88,2.07,1.59,42.150,46.740,274
2,3,1,4,3,1,2,0.86,0.86,4.13,2.07,45.450,42.150,161
3,4,1,4,4,5,1,0.87,0.86,3.60,4.13,46.200,45.450,270
4,5,1,4,5,2,5,0.81,0.87,7.43,3.60,56.085,46.200,62


[Scikitlearn](https://sklearn.org/modules/neural_networks_supervised.html) recommends to scale the data to  [0, 1] or [-1, +1] by using the StandardScaler but has an RMSE of 103 whereas QuantileTransforme which also transform data to values between [-1, +1].

In [22]:
#Split into train and test set
X_train =X.loc[0:1211]
X_test= X.loc[1212:1514]
y_train = y[0:1212]
y_test=y[1212:1514]
#use min max scaler to get values between 0 and 1
scaler = MinMaxScaler()
#Compute the minimum and maximum of X to be used for later scaling
scaler.fit(X_train)
scaler.fit(X_test)
#Scaling features of X according to feature_range
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
#convert series into array for reshaping
target_train = np.array(y_train, dtype=pd.Series)
#reshape data -> ValueError: Expected 2D array, got 1D array instead:
#array=[281. 279. 274. ...  43.  67. 393.].
#Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.
y_train_reshaped = target_train.reshape(-1, 1)
#Compute the minimum and maximum of y_train to be used for later scaling
scaler.fit(y_train_reshaped)
#Scaling features of y_train according to feature_range
y_train_scaled= scaler.transform(y_train_reshaped)

/home/kronork/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [23]:
#initialize MLPRegressor
mlp = MLPRegressor(
    hidden_layer_sizes=(10,5),  activation='logistic', solver='lbfgs', alpha=0.001, batch_size='auto',
    learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
    random_state=9, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel()
m = mlp.fit(X_train_scaled,y_train_scaled.ravel())

predictions = m.predict(X_test_scaled)

In [24]:
#Denormalize testdata
pred = predictions.reshape(-1,1)
denormalizedX = scaler.inverse_transform(pred)
denormalizedY = scaler.inverse_transform(y_train_scaled)

In [25]:
#Compare predicted and actual values
df = pd.DataFrame({'Actual': y_test, 'Predicted': denormalizedX.ravel()})  
df[200:220]

,Actual,Predicted
1412,58,53.643703
1413,59,48.943063
1414,348,236.126494
1415,344,292.091676
1416,318,290.876283
1417,312,227.387298
1418,252,261.381907
1419,32,18.117807
1420,28,40.794903
1421,307,233.866831


In [26]:
#RMSE version 1
a=df.loc[:,'Actual']
b=df.loc[:,'Predicted']
rmse=sqrt(mean_squared_error(a, b))  
"The root mean square error of the model is {0:.3f}".format(rmse) 
#'The root mean square error of the model is 79.160' -> minMax normalize + Denormalize +logistic
#'The root mean square error of the model is 86.561'-> minMax normalize + Denormalize +relu
#'The root mean square error of the model is 61.893'-> after update

'The root mean square error of the model is 61.893'

In [27]:
#Save predicted and actual values into csv
daysTmp = X.values[:,0]
day = daysTmp[1212:1514].astype(int)
df = pd.DataFrame({'Day':day,'Actual': y_test, 'Predicted': denormalizedX.ravel()})  
df.to_csv('predictionRaw4.csv')

In [28]:
#Save predicted and actual values of training data into csv
#daysTmp = X.values[:,0]
#day = daysTmp[0:1212].astype(int)
#df = pd.DataFrame({'Day':day,'Actual': y_train, 'Predicted': predictions})  
#df.to_csv('training.csv')

In [29]:
#read csv with the same length as the feature matrix for y values after day 1212
predict = pd.read_csv('prediction4.csv')
predict.head()
x = predict.values[:,0]
ynew = predict.values[:,2]

In [39]:
#read csv with the same length as the feature matrix for y values until day 1212
training = pd.read_csv('trainingCleaned.csv')
training.head()
xtrain = training.values[:,1]
ytrain = training.values[:,2]

In [38]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import SingleIntervalTicker, LinearAxis
# prepare some data
x1 = xtrain
x2 = X.values[:,0]
x3 = x

y1 = ytrain
y2 = y
y3 = ynew

# output to static HTML file
output_file("MLPRegressor.html")
p = figure(
   tools="pan,box_zoom,wheel_zoom,reset,save",
    title="MLPRegressor Prediction",
    y_range=[1, 850],
    x_range=[0, 1600],
   x_axis_label='Days', y_axis_label='Frequency',
    plot_width=2000, plot_height=350,
    x_axis_type=None,
    toolbar_location="left"
)
#set xticks
tickerx = SingleIntervalTicker(interval=1, num_minor_ticks=1)
xaxis = LinearAxis(ticker=tickerx)
p.add_layout(xaxis, 'below')
# add a line renderer with legend and line thickness
p.ray(x=1212, y=0, length=800, angle=1.57079633, color='purple')
p.line(x1, y1, legend="Predicted Training", line_color="green")
p.line(x2, y2, legend="Actual", line_color="blue")
p.line(x3, y3, legend="Predicted Test", line_color="red")
p.xaxis.major_label_orientation = 'vertical'
# show the results
show(p)